In [11]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [30]:
import time
import pickle
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display, Markdown

from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex, ResponseSynthesizer
from llama_index.data_structs.node import Node, DocumentRelationship

from langchain.chat_models import ChatOpenAI
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from llama_index import GPTListIndex
from llama_index.indices.list.retrievers import ListIndexLLMRetriever
from llama_index.indices.postprocessor import (
    LLMRerank
)
from llama_index.indices.postprocessor.node import AutoPrevNextNodePostprocessor

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.indices.postprocessor.node import PrevNextNodePostprocessor
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.storage_context import StorageContext

from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [31]:
with open('/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl', 'rb') as f:
    cl_combineddoc = pickle.load(f)

# convert to llama documents
llama_cl_combineddoc = []
for doc in cl_combineddoc:
    llama_cl_combineddoc.append(Document(text=doc.page_content, extra_info=doc.metadata))




In [32]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/algovate/data/more-chainlink-tests.csv", header=None)
df.columns = ["question", "source"]
df["answer"] = ""

In [41]:
# current_question_index = 0

# # Create text box and display the first question
# answer_box = widgets.Textarea(value=df['answer'][current_question_index], placeholder='Type your answer here', rows=3)
# question_display = widgets.HTML(value=f"<h4>Question {current_question_index + 1}: {df['question'][current_question_index]}</h4>")
# display(question_display)
# display(answer_box)

# # Function to update the DataFrame with the answer and display the next question
# def update_answer(btn):
#     global current_question_index
#     df.at[current_question_index, 'answer'] = answer_box.value  # Update the answer in the DataFrame
#     current_question_index += 1  # Move to the next question

#     if current_question_index < len(df):
#         question_display.value = f"<h4>Question {current_question_index + 1}: {df['question'][current_question_index]}</h4>"
#         answer_box.value = df['answer'][current_question_index]
#     else:
#         next_button.disabled = True
#         next_button.description = 'Answers saved!'

# # Create a button to save the answers or move to the next question
# next_button = widgets.Button(description='Next Question')
# next_button.on_click(update_answer)

# # Display the next button
# display(next_button)

In [38]:
df.to_csv("/home/marshath/play/chainlink/algovate/algovate/data/more-chainlink-tests.csv", index=False)

In [39]:
df.dropna(inplace=True)

### Base

In [16]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0,))

max_input_size = 4096
num_output = 256
max_chunk_overlap = 20

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

In [17]:
index_base_combineddoc = GPTVectorStoreIndex.from_documents(
    llama_cl_combineddoc, 
    service_context=service_context
)

query_engine_base_combineddoc = index_base_combineddoc.as_query_engine()

In [18]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        df2.loc[i, "result"] = query_engine_base_combineddoc.query(gt.question)
    except Exception as e:
        print(e)
        df2.loc[i, "result"] = "ERROR"

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3642e33aaa1ed329a618814d8ed04776 in your message.).


NameError: name 'logger' is not defined